In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import re
import joblib as jb

In [7]:
like = jb.load("like.pkl.z")
tag_like_y = jb.load("tag_like_y.pkl.z")
tag_like_num = jb.load("tag_like_num.pkl.z")

# P-valor

In [8]:
from scipy.stats import binom

In [22]:
tag_pvals = dict()
global_bias = 0.4093

for tag in tag_like_num.keys():
    k = tag_like_y[tag]
    n = tag_like_num[tag]
    tag_pvals[tag] = np.min([binom.cdf(k=k, n=n, p=global_bias), binom.sf(k=k-1, n=n, p=global_bias)])

# Bias factor ponderado com p-valor

In [23]:
def logit(pval, bias_denominator=12): 
    pval = np.max([pval, 1e-5])
    inverse_pval = 1 - pval
    logit_ = np.log(inverse_pval) - np.log(1 - inverse_pval)
    
    return np.max([0, logit_]) / bias_denominator #logit of 1-pval, menor pval = mais peso

tag_pvals_logit = dict()
for tag, pval in tag_pvals.items():
    tag_pvals_logit[tag] = logit(pval)

In [24]:
tag_bias_factor = jb.load("tag_bias_factor.pkl.z")

In [31]:
tag_bias_factor_weighted = dict()

for tag, bias in tag_bias_factor.items():
    tag_bias_factor_weighted[tag] = tag_bias_factor[tag] ** tag_pvals_logit[tag]

In [33]:
jb.dump(tag_pvals, "tag_pvals.pkl.z")
jb.dump(tag_bias_factor_weighted, "tag_bias_factor_weighted.pkl.z")

['tag_bias_factor_weighted.pkl.z']

In [32]:
tag_bias_factor_weighted

{'java': 0.5373935691091647,
 'repeat': 1.0,
 'sql': 0.2445450175101906,
 'sql-server': 0.3090015724713911,
 'optional': 1.3387019266150995,
 'javascript': 0.8502985087996958,
 'image': 1.0,
 'overlay': 1.0226869207974016,
 'react-native': 2.3102744881102186,
 'opacity': 1.0,
 'swift': 1.394493993688489,
 'operators': 0.39517429600931553,
 'whitespace': 1.0,
 'ternary-operator': 1.0,
 'android': 1.0076652337958572,
 'material-design': 1.2661688913187092,
 'floating-action-button': 1.0226869207974016,
 'c': 0.5131270418042303,
 'pointers': 0.0901667753904787,
 'data-structures': 0.7488107178301262,
 'jquery': 0.3149376551699518,
 'jquery-ui': 0.9650812499345197,
 'html': 0.3752707717601929,
 'css': 0.5114057194172309,
 'twitter-bootstrap': 0.9130773276365526,
 'php': 0.3405056958231892,
 'mysql': 0.31831248566495984,
 'codeigniter': 0.2973845357058862,
 'mysqli': 0.05631453478794368,
 'windows-10': 1.063133882846613,
 'windows-10-mobile': 1.0276862051345734,
 'windows-10-universal': 1.3